In [1]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="geoapi")
location = geolocator.geocode("Accra, Ghana", exactly_one=True)

# Bounding box information
print("Bounding box:", location.raw['boundingbox'])


Bounding box: ['5.5171908', '5.6673682', '-0.2841318', '-0.0548565']


In [6]:
import googlemaps
import requests
import pandas as pd

# Replace with your actual Google Maps API key
gmaps = googlemaps.Client(key='YOUR KEY')

# Defining Overpass API query for POIs in Accra
query = """
[out:json];
(
  node["amenity"~"bank|market|fuel_station|school|church|restaurant|hotel|bistro|hospital|clinic|stadium|beach|airport|museum|club|pub|bar|cafe|pharmacy|mosque"](5.530,-0.30,6.030,0.200);
  way["amenity"~"bank|market|fuel_station|school|church|restaurant|hotel|bistro|hospital|clinic|stadium|beach|airport|museum|club|pub|bar|cafe|pharmacy|mosque"](5.530,-0.30,6.030,0.200);
  relation["amenity"~"bank|market|fuel_station|school|church|restaurant|hotel|bistro|hospital|clinic|stadium|beach|airport|museum|club|pub|bar|cafe|pharmacy|mosque"](5.530,-0.30,6.030,0.200);
);
out body;
"""


# Getting data from Overpass API
response = requests.get("https://overpass-api.de/api/interpreter", params={'data': query})

if response.status_code == 200:
    data = response.json()
    pois = []

    for element in data.get('elements', []):
        # Default values
        latitude = None
        longitude = None

        # Handle nodes
        if element['type'] == 'node':
            latitude = element['lat']
            longitude = element['lon']

        # Handle ways and relations
        name = element.get('tags', {}).get('name', 'Unnamed')
        amenity = element.get('tags', {}).get('amenity', 'Unknown')

        # Initialize popularity
        popularity = "No Data"

        # Use Google Maps API for nodes with coordinates
        if latitude is not None and longitude is not None:
            geocode_result = gmaps.reverse_geocode((latitude, longitude))
            if geocode_result:
                place_id = geocode_result[0]['place_id']

                # Get popularity and reviews data for the POI
                place_details = gmaps.place(place_id=place_id)
                popularity = place_details.get("result", {}).get("user_ratings_total", "No Data")

        # Appending to the list
        pois.append({
            'Type': element['type'].capitalize(),
            'Latitude': latitude if latitude is not None else "N/A",
            'Longitude': longitude if longitude is not None else "N/A",
            'Name': name,
            'Amenity': amenity,
            'Popularity': popularity,
        })

    # Creating a DataFrame
    pois_df = pd.DataFrame(pois)

    # Saving the DataFrame to an Excel file
    pois_df.to_excel("accra_pois_with_popularity1.xlsx", index=False)
    print("Data saved to accra_pois_with_popularity.xlsx successfully!")
else:
    print(f"Error: Unable to fetch data. Status code: {response.status_code}")

Data saved to accra_pois_with_popularity.xlsx successfully!
